In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [13]:
month_code = '08'
month_day = 31

In [5]:
file_discern_name = month_code + '.xlsx'
ncd_app = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_app24'+file_discern_name)
ncd_att = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_att24'+file_discern_name)
ncd_box = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_box24'+file_discern_name)
ncd_corlt = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corlt24'+file_discern_name)
ncd_corstd = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstd24'+file_discern_name)
ncd_corstdp = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_corstdp24'+file_discern_name)
ncd_cort = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_cort24'+file_discern_name)
ncd_drv = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_drv24'+file_discern_name)
#ncd_cnt = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_cnt24'+file_discern_name)
ncd_fin = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_fin24'+file_discern_name)
ncd_pay = pd.read_excel('/Users/user/Documents/NCD 데이터/ncd_pay24'+file_discern_name)

ncd_app['plan'] = 'APP_STD'
ncd_att['plan'] = 'ATT_STD'
ncd_corlt['plan'] = 'COR_LT'
ncd_corstd['plan'] = 'COR_STD'
ncd_corstdp['plan'] = 'COR_STD_P'
ncd_cort['plan'] = 'COR_T'
ncd_drv['plan'] = ''
ncd_box['plan'] = 'BOX'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 체험판','plan'] = 'DRV_T'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 Lite','plan'] = 'DRV_LT'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 Standard','plan'] = 'DRV_STD'
ncd_drv.loc[ncd_drv['미터링유형코드명']=='웍스 드라이브 Standard Plus','plan'] = 'DRV_STD_P'
#ncd_cnt['plan'] = ''
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 체험판','plan'] = 'CNT_T'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Lite','plan'] = 'CNT_LT'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Business','plan'] = 'CNT_BZ'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Team','plan'] = 'CNT_TM'
#ncd_cnt.loc[ncd_cnt['미터링유형코드명']=='웍스 클로바노트 Enterprise','plan'] = 'CNT_EP'
ncd_fin['plan'] = 'FIN_STD'
ncd_pay['plan'] = 'PAY_STD'
ncd_box['사용량'] = ncd_box['상품사이즈']
ncd_box['사용량'] = ncd_box['사용량'].fillna(0)

ncd = pd.concat([ncd_app, ncd_att, ncd_box, ncd_corlt, ncd_corstd, ncd_corstdp, ncd_cort, ncd_drv, ncd_fin, ncd_pay])
sales_expectation = []
ncd_copy = ncd.copy()
for i in range(1,month_day+1) :
    ncd = ncd_copy[ncd_copy['미터링일시']<= (20240000+(int(month_code)*100)+i)]
    ncd = ncd.reset_index(drop=True)
    ncd = ncd[ncd['사용량'] != 0]
    ncd = ncd[ncd['요금'] != 0]
    ncd = ncd[ncd['회원그룹명'] != '내부회원']
    ncd['구분1'] = 'Business Application'
    ncd['구분2'] = 'Business Applications'
    ncd['구분3'] = ncd['인스턴스유형코드명']
    ncd['상품명'] = ncd['미터링유형코드명']
    ncd['플랫폼'] = '민간'
    ncd['리전명_y'] = '한국'
    ncd['리전번호_y'] = 1
    ncd['dayCount'] = ncd.groupby(['회원번호', '미터링유형코드명'])['회원번호'].transform('size')
    ncd['paid user 여부'] = ncd['요금'].apply(lambda x : 'trial' if x == 0 else 'paid')
    ncd['약정할인금액'] = np.where(
        (ncd['미터링유형코드명'].isin(['웍스 코어 Standard Plus'])) & (ncd['약정여부'] == 'Y'), 
        2000, 
        np.where(
            (ncd['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus'])) & (ncd['약정여부'] == 'Y'), 
            1000,
            np.where(
                (ncd['미터링유형코드명'].isin(['웍스 급여'])) & (ncd['약정여부'] == 'Y'), 
                1500,
                np.where(
                    (ncd['미터링유형코드명'].isin(['웍스 결재'])) & (ncd['약정여부'] == 'Y'), 
                    1000, 
                    np.where(
                        (ncd['미터링유형코드명'].isin(['웍스 근태','웍스 재무'])) & (ncd['약정여부'] == 'Y'), 
                        50,
                        np.where(
                            (ncd['미터링유형코드명'].isin(['웍스 코어 Lite','웍스 코어 Standard','웍스 코어 Standard Plus','웍스 드라이브 Lite','웍스 드라이브 Standard','웍스 드라이브 Standard Plus','웍스 결재','웍스 근태','웍스 재무','웍스 급여'])) & (ncd['약정여부'] == 'N'), 
                            0,
                            0
                        )
                    )
                )
            )
        )
    )
    ncd['미터링일시'] = ncd['미터링일시'].astype(str)
    ncd['월별일수'] = np.where(
        ncd['미터링일시'].str[4:6].isin(['01', '03', '05', '07', '08', '10', '12']),
        31,
        np.where(
            ncd['미터링일시'].str[4:6].isin(['04', '06', '09', '11']),
            30,
            np.where(
                (ncd['미터링일시'].str[0:4].astype(int) % 4 == 0),
                29, 
                28
            )
        )
    )
    
    ncd['일요금(당일)'] = (ncd['요금']/ncd['월별일수'])*ncd['사용량']
    ncd['약정할인 후 요금'] = ncd['요금'] - ncd['약정할인금액']
    ncd['약정할인율'] = ncd['약정할인금액']/ncd['요금']
    ncd['결합할인 여부'] = 'Y'
    ncd.loc[ncd['미터링유형코드명'].isin(['웍스 근태', '웍스 재무', '웍스 급여']), '결합할인 여부'] = 'N'
    ncd['결합할인 여부'] = np.where(
        (ncd.duplicated(subset=['회원번호','미터링일시','paid user 여부','약정여부','결합할인 여부'], keep=False)) & (ncd['paid user 여부'] == 'paid') & (ncd['약정여부'] == 'Y') & (ncd['결합할인 여부'] == 'Y'),
        'Y',
        'N')
    ncd.loc[ncd['약정여부'] == 'N', '결합할인 여부'] = 'N'
    ncd['미터링일시'] = ncd['미터링일시'].astype(str)
    ncd['일매출(당일)'] = np.where(
        (ncd['결합할인 여부'] == 'Y'),
        ((ncd['요금']/ncd['월별일수'])*ncd['사용량']*(1-ncd['약정할인율']))*0.9,
        (ncd['요금']/ncd['월별일수'])*ncd['사용량']*(1-ncd['약정할인율']))
    ncd = ncd.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'])
    ncd['일매출(전일)'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].shift(1)
    ncd['일매출 증가율'] = ((ncd['일매출(당일)'] - ncd['일매출(전일)']) / ncd['일매출(전일)'])
    ncd['누적 일매출'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].cumsum()
    ncd['월매출(당월)'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일매출(당일)'].transform('sum')
    ncd['월요금(당월)'] = ncd.groupby(['회원번호', '미터링유형코드명'])['일요금(당일)'].transform('sum')
    ncd['사용량_mean'] = ncd.groupby(['회원번호', '미터링유형코드명'])['사용량'].transform('mean')
    ncd = ncd.drop(['dayCount'], axis = 1)
    ncd = ncd.sort_values(by=['회원번호', '미터링유형코드명', '미터링일시'], ascending = [True, False, True])
    ncd = ncd.reset_index(drop=True)
    ncd.loc[ncd['미터링유형코드명'].str[:5] == '웍스 드라', '청구유형코드명'] = 'WORKBOX'
    ncd.loc[ncd['미터링유형코드명'].isin(['웍스 결재','웍스 근태','웍스 재무','웍스 급여']), '청구유형코드명'] = 'WORKPLACE'
    ncd = ncd[['회원번호','파트너사회원번호','회원그룹코드','청구유형코드명','미터링유형코드명','plan','미터링일시','사용량_mean','약정여부','월매출(당월)','월요금(당월)']]
    ncd_unique = ncd.drop_duplicates(subset=['회원번호', '미터링유형코드명'], keep='first').reset_index(drop=True)
    ncd_unique['회원번호'] = ncd_unique['회원번호'].astype(int)
    ncd_unique.columns = ['회원번호','파트너사회원번호','회원그룹코드','구분3','미터링유형코드명','plan','미터링일시','사용량','약정여부','월매출(당월)','월요금(당월)']
    ncd_unique1 = ncd_unique.groupby(['회원번호','회원그룹코드','구분3'])[['월매출(당월)','월요금(당월)']].sum().reset_index()
    ncd_unique1 = pd.merge(ncd_unique1, ncd_unique[['회원번호','파트너사회원번호','구분3','plan','사용량']], how = 'left',on = ['회원번호','구분3'])
    ncd_unique = ncd_unique1
    ncd_unique = ncd_unique.drop_duplicates(subset=['회원번호', '구분3'], keep='last').reset_index(drop=True)
    delete1 = pd.read_excel('/Users/user/Documents/업무문서/iima 테스트도메인.xlsx')
    delete2 = pd.read_excel('/Users/user/Documents/업무문서/Edu 제외.xlsx')
    delete3 = pd.read_excel('/Users/user/Documents/업무문서/파트너 제외.xlsx')
    delete4 = pd.read_excel('/Users/user/Documents/업무문서/수동 제외.xlsx')
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete1['ncpId'])]
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete2['ncpId'])]
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete3['partnerDomainId'])]
    ncd_unique = ncd_unique[~ncd_unique['회원번호'].isin(delete4['ncpId'])]
    pt = pd.read_excel('/Users/user/Documents/업무문서/pt8.xlsx')
    merge = pd.merge(ncd_unique, pt, how='left', on=['회원번호', '구분3', '파트너사회원번호'])
    merge.columns = ['회원번호','회원그룹코드','구분3','예상월매출','예상월요금','파트너사회원번호','plan','사용량','파트너할인율']
    merge['파트너할인율'] = merge['파트너할인율'].fillna(0)
    merge['예상월매출'] = merge['예상월매출'] * (1 - merge['파트너할인율'])
    merge = merge[['회원번호','회원그룹코드','파트너사회원번호','구분3','plan','예상월매출','예상월요금','사용량']]
    merge = merge.drop_duplicates(subset=['회원번호', '구분3'], keep='last').reset_index(drop=True)
    pt2 = pd.read_excel('/Users/user/Documents/업무문서/pt8_2.xlsx')
    sales8 = pd.read_excel('/Users/user/Documents/업무문서/ncd8_sales_expectation.xlsx')
    merge_original = merge.merge(sales8[['회원번호', '구분3']], on=['회원번호', '구분3'], how='inner')
    merge_new = merge.merge(sales8[['회원번호', '구분3']], on=['회원번호', '구분3'], how='outer', indicator=True)
    merge_new = merge_new[merge_new['_merge'] == 'left_only'].drop(columns=['_merge'])
    merge_new = pd.merge(merge_new, pt2, how='left', on=['파트너사회원번호', '구분3'])
    merge_new['파트너할인율'] = merge_new['파트너할인율'].fillna(0)
    merge_new['예상월매출'] = merge_new['예상월매출'] * (1 - merge_new['파트너할인율'])
    merge = pd.concat([merge_original, merge_new])
    merge = merge[['회원번호','회원그룹코드','파트너사회원번호','구분3','plan','예상월매출','예상월요금','사용량']]
    merge = merge.drop_duplicates(subset=['회원번호', '구분3'], keep='last').reset_index(drop=True)
    
    promotion = pd.read_excel('/Users/user/Documents/진행중 업무/웍스오픈프로모션_24.06-09대상자_241010_v3.xlsx', sheet_name=None)
    june = list(promotion.keys())[1]
    june_promotion = promotion[june]
    june_promotion = june_promotion.iloc[1:8,0:6]
    june_promotion['회원번호'] = june_promotion['회원번호'].astype(int)
    july = list(promotion.keys())[2]
    july_promotion = promotion[july]
    july_promotion = july_promotion.iloc[1:,0:6]
    july_promotion = july_promotion.drop(411, axis=0)
    july_promotion['회원번호'] = july_promotion['회원번호'].astype(int)
    august = list(promotion.keys())[3]
    august_promotion = promotion[august]
    august_promotion = august_promotion.iloc[1:,0:6]
    august_promotion['회원번호'] = august_promotion['회원번호'].astype(int)
    september = list(promotion.keys())[4]
    september_promotion = promotion[september]
    september_promotion = september_promotion.iloc[1:,0:6]
    september_promotion['회원번호'] = september_promotion['회원번호'].astype(int)
    
    # june_promotion 제외
    temp_merge = merge.merge(june_promotion[['회원번호', 'base_item_id 상품명']], 
                             left_on=['회원번호', 'plan'], 
                             right_on=['회원번호', 'base_item_id 상품명'], 
                             how='left', indicator='_merge1')
    merge = temp_merge[temp_merge['_merge1'] == 'left_only'].drop(columns=['_merge1', 'base_item_id 상품명'])
    
    # july_promotion 제외
    temp_merge = merge.merge(july_promotion[['회원번호', 'base_item_id 상품명']], 
                             left_on=['회원번호', 'plan'], 
                             right_on=['회원번호', 'base_item_id 상품명'], 
                              how='left', indicator='_merge2')
    merge = temp_merge[temp_merge['_merge2'] == 'left_only'].drop(columns=['_merge2', 'base_item_id 상품명'])
    
    # august_promotion 제외
    #temp_merge = merge.merge(august_promotion[['회원번호', 'base_item_id 상품명']], 
    #                         left_on=['회원번호', 'plan'],
    #                         right_on=['회원번호', 'base_item_id 상품명'], 
    #                         how='left', indicator='_merge3')
    #merge = temp_merge[temp_merge['_merge3'] == 'left_only'].drop(columns=['_merge3', 'base_item_id 상품명'])
    
    # september_promotion 제외
    #temp_merge = merge.merge(september_promotion[['회원번호', 'base_item_id 상품명']], 
    #                         left_on=['회원번호', 'plan'], 
    #                         right_on=['회원번호', 'base_item_id 상품명'],
    #                         how='left', indicator='_merge4')
    #merge = temp_merge[temp_merge['_merge4'] == 'left_only'].drop(columns=['_merge4', 'base_item_id 상품명'])
    
    total_sales = merge['예상월매출'].sum() * month_day / i
    sales_expectation.append(total_sales)

In [11]:
sales_expectation

[2021253008.95204,
 2021261070.4081976,
 2020418826.1538393,
 2019690133.419963,
 2019304553.580978,
 2019133753.7332964,
 2019490845.5582654,
 2019864263.9672444,
 2020128723.7354536,
 2020471123.2400346,
 2020841041.762934,
 2020968477.6731298,
 2021380365.7971447,
 2021764283.423103,
 2022130156.063666,
 2022262238.3009455,
 2022596533.6464107,
 2023380138.8983638,
 2023483629.2793508,
 2023602414.0694032,
 2023869762.5053508,
 2024149595.219567,
 2024676132.5026565,
 2025005733.3210535,
 2025327026.2064648,
 2025618890.1608484,
 2025906784.048355,
 2026221371.8796864,
 2026531106.8643348,
 2026914977.3883903,
 2027179022.9781954]

In [11]:
merge.to_excel('/Users/user/Documents/업무문서/ncd8_sales_expectation.xlsx',index=False)

In [113]:
ncd9 = pd.read_excel('/Users/user/Documents/업무문서/ncd9_sales_expectation.xlsx')
ncd10 = pd.read_excel('/Users/user/Documents/업무문서/ncd10_sales_expectation.xlsx')

In [114]:
n = pd.merge(ncd10, ncd9, how = 'outer', on = ['회원번호','구분3'])

In [77]:
n.to_excel('/Users/user/Documents/업무문서/ncd9_10_sales_expectation.xlsx',index=False)

In [117]:
boa9 = pd.read_excel('/Users/user/Documents/BOA 데이터/boa2409.xlsx')

In [118]:
m = pd.merge(ncd9, boa9, how = 'right', on = ['회원번호','구분3'])

In [119]:
m.to_excel('/Users/user/Documents/업무문서/ncd9_boa9_sales_expectation.xlsx',index=False)